# LoRA for token classification

Low-Rank Adaptation (LoRA) is a reparametrization method that aims to reduce the number of trainable parameters with low-rank representations. The weight matrix is broken down into low-rank matrices that are trained and updated. All the pretrained model parameters remain frozen. After training, the low-rank matrices are added back to the original weights. This makes it more efficient to store and train a LoRA model because there are significantly fewer parameters.
> <font style="color:darkgreen">💡 Read [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685) to learn more about LoRA.</font>

This guide will show you how to train a [`roberta-large`](https://huggingface.co/roberta-large) model with LoRA on the [BioNLP2004](https://huggingface.co/datasets/tner/bionlp2004) dataset for token classification.

Before you begin, make sure you have all the necessary libraries installed:

```
!pip install -q peft transformers datasets evaluate seqeval
```

## Setup
Let's start by importing all the necessary libraries youll need:
- 🤗 Transformers for loading the base `roberta-large model and tokenizer, and handling the training loop
- 🤗 Datasets for loading and preparing the `bionlp2004` dataset for training
- 🤗 Evaluate for evaluating the model's performance
- 🤗 PEFT for setting up the LoRA configuration and creating the PEFT model

In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

model_checkpoint = "roberta-large"
lr = 1e-3
batch_size = 16
num_epochs = 10

## Load dataset and metric
The [BioNLP2004](https://huggingface.co/datasets/tner/bionlp2004) dataset includes tokens and tags for biological structures like DNA, RNA and proteins. Load the dataset:

In [2]:
bionlp = load_dataset("tner/bionlp2004")
bionlp["train"][0]

{'tokens': ['Since',
  'HUVECs',
  'released',
  'superoxide',
  'anions',
  'in',
  'response',
  'to',
  'TNF',
  ',',
  'and',
  'H2O2',
  'induces',
  'VCAM-1',
  ',',
  'PDTC',
  'may',
  'act',
  'as',
  'a',
  'radical',
  'scavenger',
  '.'],
 'tags': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

The `tags` values are defined in the label ids [dictionary](https://huggingface.co/datasets/tner/bionlp2004#label-id). The letter that prefixes each label indicates the token position: `B` is for the first token of an entity, `I` is for a token inside the entity, and `0` is for a token that is not part of an entity.

```
{
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}
```

Then load the [`seqeval`](https://huggingface.co/spaces/evaluate-metric/seqeval) framework which includes several metrics - precision, accuracy, F1, and recall - for evaluating sequence labeling tasks.

In [3]:
seqeval = evaluate.load("seqeval")

Now you can write an evaluation function to compute the metrics from the model predictions and labels, and return the precision, recall, $F_1$, and accuracy scores:

In [4]:
label_list = [
    "O",
    "B-DNA",
    "I-DNA",
    "B-protein",
    "I-protein",
    "B-cell_type",
    "I-cell_type",
    "B-cell_line",
    "I-cell_line",
    "B-RNA",
    "I-RNA",
]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Preprocess dataset
Initialize a tokenizer and make sure you set `is_split_into_words=True` because the text sequence has already been split into words. However, this doesn't mean it is tokenized yet (even though it may look like it!), and you'll need to further tokenize the words into subwords.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

You’ll also need to write a function to:
1. Map each token to their respective word with the [`word_ids`](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
1. Ignore the special tokens by setting them to `-100`.
1.  Label the first token of a given entity.

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Use [`map`](https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/main_classes#datasets.Dataset.map) to apply the `tokenize_and_align_labels` function to the dataset:

In [7]:
tokenized_bionlp = bionlp.map(tokenize_and_align_labels, batched=True)
tokenized_bionlp

Map:   0%|          | 0/1927 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16619
    })
    validation: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1927
    })
    test: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3856
    })
})

Finally, create a data collator to pad the examples to the longest length in a batch:

In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
data_collator

DataCollatorForTokenClassification(tokenizer=RobertaTokenizerFast(name_or_path='roberta-large', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, 

## Train
Now you're ready to create a [PeftModel](https://huggingface.co/docs/peft/v0.8.2/en/package_reference/peft_model#peft.PeftModel). Start by loading the base `roberta-large` model, the number of expected labels, and the `id2label` and `label2id` dictionaries:

In [9]:
id2label = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-protein",
    4: "I-protein",
    5: "B-cell_type",
    6: "I-cell_type",
    7: "B-cell_line",
    8: "I-cell_line",
    9: "B-RNA",
    10: "I-RNA",
}
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=11,
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the [LoraConfig](https://huggingface.co/docs/peft/v0.8.2/en/package_reference/lora#peft.LoraConfig) with:
- `task_type`, token classification (`TaskType.TOKEN_CLS`)
- `r`, the dimension of the low-rank matrices
- `lora_alpha`, scaling factor for the weight matrices
- `lora_dropout`, dropout probability of the LoRA layers
- `bias`, set to `all` to train all bias parameters

> <font style="color:darkgreen">💡 The weight matrix is scaled by `lora_alpha/r`, and a higher `lora_alpha` value assigns more weight to the LoRA activations. For performance, we recommend setting `bias` to `None` first, and then `lora_only`, before trying `all`.</font>

In [10]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="all"
)

Pass the base model and `peft_config` to the [`get_peft_model()`](https://huggingface.co/docs/peft/v0.8.2/en/package_reference/peft_model#peft.get_peft_model) function to create a [PeftModel](https://huggingface.co/docs/peft/v0.8.2/en/package_reference/peft_model#peft.PeftModel). You can check out how much more efficient training the PeftModel is compared to fully training the base model by printing out the trainable parameters:

In [11]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,855,499 || all params: 355,905,558 || trainable%: 0.521345890304978


From the 🤗 Transformers library, create a [TrainingArguments](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/trainer#transformers.TrainingArguments) class and specify where you want to save the model to, the training hyperparameters, how to evaluate the model, and when to save the checkpoints:

In [12]:
training_args = TrainingArguments(
    output_dir="ner/logs/roberta-large-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

Pass the model, `TrainingArguments`, datasets, tokenizer, data collator and evaluation function to the [Trainer](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/trainer#transformers.Trainer) class. The `Trainer` handles the training loop for you, and when you're ready, call [`train`](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/trainer#transformers.Trainer.train) to begin!

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    #report_to=None,
    train_dataset=tokenized_bionlp["train"],
    eval_dataset=tokenized_bionlp["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.227100,0.185784,0.722738,0.756708,0.739333,0.938277
2,0.191300,0.168251,0.750610,0.775077,0.762647,0.942547
3,0.173500,0.170494,0.756673,0.791284,0.773592,0.945026
4,0.170900,0.163178,0.774877,0.796506,0.785543,0.946875
5,0.152400,0.157800,0.788320,0.828921,0.808111,0.949217
6,0.142600,0.156923,0.789873,0.825860,0.807465,0.949374
7,0.129800,0.152439,0.794409,0.823879,0.808876,0.950083
8,0.113100,0.151126,0.787056,0.845309,0.815143,0.951519
9,0.103800,0.152919,0.795754,0.837025,0.815868,0.952286
10,0.095100,0.159270,0.797976,0.837925,0.817463,0.952306


TrainOutput(global_step=10390, training_loss=0.15348631204379304, metrics={'train_runtime': 3498.6853, 'train_samples_per_second': 47.501, 'train_steps_per_second': 2.97, 'total_flos': 2.3557525146368532e+16, 'train_loss': 0.15348631204379304, 'epoch': 10.0})

## Share model
Once training is complete, you can store and share your model on the Hub if you'd like. Log in to your HuggingFace account and enter your token when prompted:

In [14]:
from huggingface_hub import notebook_login
notebook_login()

Upload the model to a specific model repository on the Hub with the [`push_to_hub`](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/model#transformers.PreTrainedModel.push_to_hub) method:

In [15]:
model.push_to_hub("mdroth/roberta-large-lora-token-classification")

HfHubHTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/api/models/mdroth/roberta-large-lora-token-classification/preupload/main (Request ID: Root=1-65e1f44f-44561d054d3533c5319d9d05;d700b6b7-144a-4206-ae11-dc53f4982f09)

Forbidden: you must use a write token to upload to a repository.

##  Inference
To use your model for inference, load the configuration and model:

In [ ]:
peft_model_id = "mdroth/roberta-large-lora-token-classification"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForTokenClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

Get some text to tokenize:

In [ ]:
text = "The activation of IL-2 gene expression and NF-kappa B through CD28 requires reactive oxygen production by 5-lipoxygenase."
inputs = tokenizer(text, return_tensors="pt")
inputs

Pass the inputs to the model, and print out the model prediction for each token:

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits
tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

<font style="font-weight:300">✔</font>